In [ ]:
!pip install torch torchvision torchaudio pandas numpy scikit-learn matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
csv_path = '/content/gdrive/MyDrive/Colab Notebooks/ML/all_auto_first_90000.csv'

df = pd.read_csv(csv_path)

print("Размер датасета:", df.shape)
print("Первые строки:")
print(df.head())

Размер датасета: (89378, 26)
Первые строки:
          bodyType brand   color fuelType  modelDate                   name  \
0            Седан  AUDI  040001   бензин     1990.0  2.8 MT (174 л.с.) 4WD   
1            Седан  AUDI  EE1D19   бензин     1982.0       1.8 MT (90 л.с.)   
2  Универсал 5 дв.  AUDI  0000CC   бензин     1988.0  2.3 MT (136 л.с.) 4WD   
3            Седан  AUDI  CACECB   бензин     1988.0       1.8 MT (90 л.с.)   
4            Седан  AUDI  040001   бензин     1990.0      2.0 MT (101 л.с.)   

   numberOfDoors  productionDate          vehicleConfiguration  \
0            4.0            1991          SEDAN MECHANICAL 2.8   
1            4.0            1986          SEDAN MECHANICAL 1.8   
2            5.0            1989  WAGON_5_DOORS MECHANICAL 2.3   
3            4.0            1989          SEDAN MECHANICAL 1.8   
4            4.0            1991          SEDAN MECHANICAL 2.0   

  vehicleTransmission  ...  Руль  Состояние Владельцы        ПТС Таможня  \
0       

In [ ]:
print("\nНАЧИНАЕМ ОБРАБОТКУ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...")
print("=" * 60)

print("1. УДАЛЕНИЕ ПОЛНОСТЬЮ ПУСТЫХ СТОЛБЦОВ:")
empty_columns = ['Состояние', 'hidden']

for col in empty_columns:
    if col in df.columns:
        print(f"  Удаляем столбец: {col}")
        df = df.drop(columns=[col], errors='ignore')

print("\n2. ОБРАБОТКА ЧИСЛОВЫХ ПРИЗНАКОВ:")
numeric_columns = df.select_dtypes(include=[np.number]).columns

for col in numeric_columns:
    if df[col].isnull().any():
        before = df[col].isnull().sum()
        if col == 'price':
            df = df.dropna(subset=[col])
            after = df[col].isnull().sum()
            print(f"  {col}: строки с пропусками удалены ({before} -> {after})")
        else:
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)
            after = df[col].isnull().sum()
            print(f"  {col}: {before} -> {after} (заполнено медианой)")

print("\n3. ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ:")
categorical_columns = df.select_dtypes(include=['object']).columns

for col in categorical_columns:
    if df[col].isnull().any():
        before = df[col].isnull().sum()

        if col == 'description':
            df[col] = df[col].fillna('')
        elif col == 'Владение':
            df[col] = df[col].fillna('Не указано')
        else:
            if not df[col].mode().empty:
                mode_value = df[col].mode()[0]
                df[col] = df[col].fillna(mode_value)
            else:
                df[col] = df[col].fillna('Неизвестно')

        after = df[col].isnull().sum()
        print(f"  {col}: {before} -> {after} (обработано)")

print("\n4. УДАЛЕНИЕ СТРОК С ПРОПУСКАМИ В КЛЮЧЕВЫХ ПРИЗНАКАХ:")
key_columns_to_check = [
    'bodyType', 'modelDate', 'name', 'numberOfDoors',
    'vehicleConfiguration', 'vehicleTransmission',
    'engineDisplacement', 'enginePower', 'Привод'
]

initial_rows = len(df)
for col in key_columns_to_check:
    if col in df.columns and df[col].isnull().any():
        before_rows = len(df)
        df = df.dropna(subset=[col])
        after_rows = len(df)
        print(f"  {col}: удалено {before_rows - after_rows} строк")

print(f"\nВсего удалено строк: {initial_rows - len(df)}")
print("Текущий размер датасета:", df.shape)



НАЧИНАЕМ ОБРАБОТКУ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ...
1. УДАЛЕНИЕ ПОЛНОСТЬЮ ПУСТЫХ СТОЛБЦОВ:
  Удаляем столбец: Состояние
  Удаляем столбец: hidden

2. ОБРАБОТКА ЧИСЛОВЫХ ПРИЗНАКОВ:
  modelDate: 1 -> 0 (заполнено медианой)
  numberOfDoors: 1 -> 0 (заполнено медианой)
  enginePower: 1 -> 0 (заполнено медианой)
  Владельцы: 13588 -> 0 (заполнено медианой)
  price: строки с пропусками удалены (410 -> 0)

3. ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ:
  bodyType: 1 -> 0 (обработано)
  name: 1 -> 0 (обработано)
  vehicleConfiguration: 1 -> 0 (обработано)
  vehicleTransmission: 1 -> 0 (обработано)
  engineDisplacement: 1 -> 0 (обработано)
  description: 3234 -> 0 (обработано)
  Привод: 1 -> 0 (обработано)
  ПТС: 361 -> 0 (обработано)
  Владение: 59923 -> 0 (обработано)

4. УДАЛЕНИЕ СТРОК С ПРОПУСКАМИ В КЛЮЧЕВЫХ ПРИЗНАКАХ:

Всего удалено строк: 0
Текущий размер датасета: (88968, 24)


In [ ]:
print("\nПРЕОБРАЗОВАНИЕ ПОЛЕЙ...")
print("=" * 60)

if 'description' in df.columns:
    print("Преобразуем 'description' в 'description_length'...")
    df['description_length'] = df['description'].apply(len)
    df = df.drop(columns=['description'])

if 'engineDisplacement' in df.columns:
    print("Преобразуем 'engineDisplacement' в число...")
    df['engineDisplacement'] = df['engineDisplacement'].astype(str).str.extract(r'(\d+\.\d+|\d+)')
    df['engineDisplacement'] = df['engineDisplacement'].astype(float)

if 'start_date' in df.columns:
    print("Обрабатываем 'start_date'...")
    try:
        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        successful = df['start_date'].notna().sum()
        print(f"  Успешно преобразовано дат: {successful}/{len(df)}")
        if successful > 0:
            df['start_year'] = df['start_date'].dt.year
            df['start_month'] = df['start_date'].dt.month
            df['start_day'] = df['start_date'].dt.day
        df = df.drop(columns=['start_date'])
    except Exception as e:
        print("  Ошибка при обработке 'start_date':", e)
        df = df.drop(columns=['start_date'], errors='ignore')

print("\nТекущие столбцы:")
print(df.columns.tolist())
print("Текущий размер датасета:", df.shape)



ПРЕОБРАЗОВАНИЕ ПОЛЕЙ...
Преобразуем 'description' в 'description_length'...
Преобразуем 'engineDisplacement' в число...
Обрабатываем 'start_date'...
  Успешно преобразовано дат: 88967/88968

Текущие столбцы:
['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement', 'enginePower', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'model', 'description_length', 'start_year', 'start_month', 'start_day']
Текущий размер датасета: (88968, 26)


In [ ]:
print("\nКОДИРОВАНИЕ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ...")
print("=" * 60)

categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}

print(f"Категориальные признаки ({len(categorical_columns)}):")
for col in categorical_columns:
    if col == 'price':
        continue
    print(f"  {col} (уникальных: {df[col].nunique()})")
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le
    print("    ✅ Закодирован")

print("✅ Все категориальные признаки закодированы.")

print("\nУДАЛЕНИЕ ПРОБЛЕМНЫХ СТОЛБЦОВ...")
print("=" * 60)

columns_to_drop = []
for col in df.columns:
    if col == 'price':
        continue
    nunique = df[col].nunique()
    if nunique <= 1:
        columns_to_drop.append(col)
        print(f"  {col} удалён (константа)")
    elif nunique > len(df) * 0.8:
        columns_to_drop.append(col)
        print(f"  {col} удалён (уникальных слишком много: {nunique})")

if columns_to_drop:
    df = df.drop(columns=columns_to_drop)

print("\nИтоговые столбцы:")
print(df.columns.tolist())
print("Размер данных:", df.shape)



КОДИРОВАНИЕ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ...
Категориальные признаки (13):
  bodyType (уникальных: 168)
    ✅ Закодирован
  brand (уникальных: 36)
    ✅ Закодирован
  color (уникальных: 16)
    ✅ Закодирован
  fuelType (уникальных: 6)
    ✅ Закодирован
  name (уникальных: 4484)
    ✅ Закодирован
  vehicleConfiguration (уникальных: 1860)
    ✅ Закодирован
  vehicleTransmission (уникальных: 4)
    ✅ Закодирован
  Комплектация (уникальных: 5827)
    ✅ Закодирован
  Привод (уникальных: 3)
    ✅ Закодирован
  Руль (уникальных: 2)
    ✅ Закодирован
  ПТС (уникальных: 2)
    ✅ Закодирован
  Владение (уникальных: 291)
    ✅ Закодирован
  model (уникальных: 1086)
    ✅ Закодирован
✅ Все категориальные признаки закодированы.

УДАЛЕНИЕ ПРОБЛЕМНЫХ СТОЛБЦОВ...
  Таможня удалён (константа)

Итоговые столбцы:
['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement', 'enginePower', 'mileage', 'Ком

In [ ]:
print("\nРАЗДЕЛЕНИЕ НА ПРИЗНАКИ И ЦЕЛЬ...")
print("=" * 60)

if 'price' not in df.columns:
    raise ValueError("В датасете нет столбца 'price'!")

X = df.drop('price', axis=1)
y = df['price']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"Обучающая выборка: {X_train.shape}")
print(f"Тестовая выборка: {X_test.shape}")

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

feature_names = X.columns.tolist()
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

feature_defaults = X.median()
with open('feature_defaults.pkl', 'wb') as f:
    pickle.dump(feature_defaults.to_dict(), f)

print("\n✅ Сохранены 'label_encoders.pkl', 'feature_names.pkl', 'feature_defaults.pkl'")




РАЗДЕЛЕНИЕ НА ПРИЗНАКИ И ЦЕЛЬ...
X shape: (88968, 24)
y shape: (88968,)
Обучающая выборка: (71174, 24)
Тестовая выборка: (17794, 24)

✅ Сохранены 'label_encoders.pkl', 'feature_names.pkl', 'feature_defaults.pkl'


In [ ]:
print("\nПРОВЕРКА И ОЧИСТКА X_train / X_test ОТ NaN И inf...")
print("=" * 60)

X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

print("Всего NaN в X_train:", X_train.isna().sum().sum())
print("Всего NaN в X_test:", X_test.isna().sum().sum())

for col in X_train.columns:
    if X_train[col].isnull().any() or X_test[col].isnull().any():
        med = X_train[col].median()
        X_train[col] = X_train[col].fillna(med)
        X_test[col] = X_test[col].fillna(med)
        print(f"  Столбец '{col}': пропуски заполнены медианой ({med})")

print("\nПосле заполнения:")
print("Всего NaN в X_train:", X_train.isna().sum().sum())
print("Всего NaN в X_test:", X_test.isna().sum().sum())

print("NaN в y_train:", y_train.isna().sum())
print("NaN в y_test:", y_test.isna().sum())



ПРОВЕРКА И ОЧИСТКА X_train / X_test ОТ NaN И inf...
Всего NaN в X_train: 2691
Всего NaN в X_test: 673
  Столбец 'engineDisplacement': пропуски заполнены медианой (2.0)
  Столбец 'start_year': пропуски заполнены медианой (2020.0)
  Столбец 'start_month': пропуски заполнены медианой (8.0)
  Столбец 'start_day': пропуски заполнены медианой (14.0)

После заполнения:
Всего NaN в X_train: 0
Всего NaN в X_test: 0
NaN в y_train: 0
NaN в y_test: 0


In [ ]:
print("\nПОДГОТОВКА ДАННЫХ ДЛЯ FCN (train/val/test)...")
print("=" * 60)

from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=42
)

print("X_tr:", X_tr.shape, "y_tr:", y_tr.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

scaler = StandardScaler()
X_tr_scaled = scaler.fit_transform(X_tr)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Признаки масштабированы StandardScaler (по train).")

y_tr_log = np.log1p(y_tr.values)
y_val_log = np.log1p(y_val.values)
y_test_log = np.log1p(y_test.values)

print("✅ Целевая переменная преобразована через log1p.")

X_tr_tensor = torch.tensor(X_tr_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

y_tr_tensor = torch.tensor(y_tr_log, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val_log, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test_log, dtype=torch.float32).view(-1, 1)

batch_size = 1024
train_dataset = TensorDataset(X_tr_tensor, y_tr_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("✅ DataLoader'ы для train/val/test созданы.")

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✅ Scaler сохранён в 'scaler.pkl'.")



ПОДГОТОВКА ДАННЫХ ДЛЯ FCN (train/val/test)...
X_tr: (56939, 24) y_tr: (56939,)
X_val: (14235, 24) y_val: (14235,)
X_test: (17794, 24) y_test: (17794,)
✅ Признаки масштабированы StandardScaler (по train).
✅ Целевая переменная преобразована через log1p.
✅ DataLoader'ы для train/val/test созданы.
✅ Scaler сохранён в 'scaler.pkl'.


In [ ]:
print("\nСОЗДАЁМ ОБНОВЛЁННУЮ FCN-МОДЕЛЬ...")
print("=" * 60)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")

class SimpleFCNRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

input_dim = X_tr.shape[1]
model_fcn = SimpleFCNRegressor(input_dim=input_dim).to(device)

criterion = nn.SmoothL1Loss()

optimizer = torch.optim.AdamW(
    model_fcn.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)

print("✅ Модель и оптимизатор инициализированы.")



СОЗДАЁМ ОБНОВЛЁННУЮ FCN-МОДЕЛЬ...
Используем устройство: cpu
✅ Модель и оптимизатор инициализированы.


In [ ]:
print("\nНАЧИНАЕМ ОБУЧЕНИЕ FCN v2...")
print("=" * 60)

epochs = 200
patience = 20
best_val_loss = float('inf')
best_state_dict = None
no_improve_epochs = 0

for epoch in range(1, epochs + 1):
    model_fcn.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        preds = model_fcn(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)

    train_loss /= len(train_loader.dataset)

    model_fcn.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            preds = model_fcn(X_batch)
            loss = criterion(preds, y_batch)
            val_loss += loss.item() * X_batch.size(0)

    val_loss /= len(val_loader.dataset)


    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        best_state_dict = {k: v.cpu().clone() for k, v in model_fcn.state_dict().items()}
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1

    if epoch == 1 or epoch % 10 == 0:
        print(f"Эпоха {epoch:03d} | Train SmoothL1 (log): {train_loss:.4f} | Val SmoothL1 (log): {val_loss:.4f} | no_improve: {no_improve_epochs}")

    if no_improve_epochs >= patience:
        print(f"\n⏹ Early stopping на эпохе {epoch} (нет улучшения {patience} эпох).")
        break

print("\n✅ Обучение завершено.")
print(f"Лучшая валид. SmoothL1 (log): {best_val_loss:.4f}")

if best_state_dict is not None:
    model_fcn.load_state_dict(best_state_dict)
    model_fcn.to(device)
else:
    print("⚠️ best_state_dict = None — используются веса последней эпохи.")



НАЧИНАЕМ ОБУЧЕНИЕ FCN v2...
Эпоха 001 | Train SmoothL1 (log): 12.5305 | Val SmoothL1 (log): 11.7543 | no_improve: 0
Эпоха 010 | Train SmoothL1 (log): 0.3167 | Val SmoothL1 (log): 0.3058 | no_improve: 0
Эпоха 020 | Train SmoothL1 (log): 0.1135 | Val SmoothL1 (log): 0.1107 | no_improve: 0
Эпоха 030 | Train SmoothL1 (log): 0.0589 | Val SmoothL1 (log): 0.0606 | no_improve: 0
Эпоха 040 | Train SmoothL1 (log): 0.0467 | Val SmoothL1 (log): 0.0493 | no_improve: 0
Эпоха 050 | Train SmoothL1 (log): 0.0418 | Val SmoothL1 (log): 0.0442 | no_improve: 0
Эпоха 060 | Train SmoothL1 (log): 0.0385 | Val SmoothL1 (log): 0.0416 | no_improve: 2
Эпоха 070 | Train SmoothL1 (log): 0.0365 | Val SmoothL1 (log): 0.0395 | no_improve: 2
Эпоха 080 | Train SmoothL1 (log): 0.0349 | Val SmoothL1 (log): 0.0380 | no_improve: 0
Эпоха 090 | Train SmoothL1 (log): 0.0336 | Val SmoothL1 (log): 0.0370 | no_improve: 1
Эпоха 100 | Train SmoothL1 (log): 0.0325 | Val SmoothL1 (log): 0.0356 | no_improve: 0
Эпоха 110 | Train Smoot

In [ ]:
print("\nОЦЕНКА КАЧЕСТВА FCN v2 НА ТЕСТЕ...")
print("=" * 60)

model_fcn.eval()
with torch.no_grad():
    y_test_pred_log = model_fcn(X_test_tensor.to(device)).cpu().numpy().squeeze()

y_test_pred = np.expm1(y_test_pred_log)
y_true = y_test.values

mae = mean_absolute_error(y_true, y_test_pred)
mse = mean_squared_error(y_true, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_test_pred)
mape = np.mean(np.abs((y_true - y_test_pred) / y_true)) * 100

within_15_percent = np.mean(np.abs((y_true - y_test_pred) / y_true) <= 0.15) * 100
within_20_percent = np.mean(np.abs((y_true - y_test_pred) / y_true) <= 0.20) * 100

print("\n📊 РЕЗУЛЬТАТЫ НА ТЕСТЕ (FCN v2):")
print("=" * 40)
print(f"MAE:  {mae:,.0f} руб.")
print(f"RMSE: {rmse:,.0f} руб.")
print(f"R²:   {r2:.3f}")
print(f"MAPE: {mape:.1f}%")
print(f"Точность в пределах 15%: {within_15_percent:.1f}%")
print(f"Точность в пределах 20%: {within_20_percent:.1f}%")



ОЦЕНКА КАЧЕСТВА FCN v2 НА ТЕСТЕ...

📊 РЕЗУЛЬТАТЫ НА ТЕСТЕ (FCN v2):
MAE:  198,942 руб.
RMSE: 775,954 руб.
R²:   0.836
MAPE: 18.1%
Точность в пределах 15%: 57.3%
Точность в пределах 20%: 69.0%


In [ ]:
import torch

torch.save(model_fcn.state_dict(), 'fcn_car_price_model_v2.pth')
print("✅ Модель v2 сохранена в 'fcn_car_price_model_v2.pth'.")


✅ Модель v2 сохранена в 'fcn_car_price_model_v2.pth'.


In [ ]:
def predict_car_price_fcn(car_features: dict) -> float:
    """
    Предсказание цены автомобиля FCN v2.

    car_features: словарь с исходными признаками автомобиля.
                  Названия ключей должны совпадать с колонками исходного df
                  (после наших преобразований: description -> description_length и т.п.)
    """
    import numpy as np
    import pandas as pd
    import torch
    import torch.nn as nn
    import pickle

    with open('label_encoders.pkl', 'rb') as f:
        loaded_encoders = pickle.load(f)

    with open('feature_names.pkl', 'rb') as f:
        feature_names = pickle.load(f)

    with open('feature_defaults.pkl', 'rb') as f:
        feature_defaults = pickle.load(f)

    with open('scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)


    try:
        with open('target_stats.pkl', 'rb') as f:
            target_stats = pickle.load(f)
        log_min = target_stats["log_min"]
        log_max = target_stats["log_max"]
    except FileNotFoundError:
        log_min = 9.0
        log_max = 19.0
        print("⚠️ target_stats.pkl не найден, используем дефолтный диапазон log_min/log_max.")

    class SimpleFCNRegressor(nn.Module):
        def __init__(self, input_dim):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, 128),
                nn.ReLU(),

                nn.Linear(128, 64),
                nn.ReLU(),

                nn.Linear(64, 32),
                nn.ReLU(),

                nn.Linear(32, 1)
            )

        def forward(self, x):
            return self.net(x)

    input_dim = len(feature_names)
    model = SimpleFCNRegressor(input_dim=input_dim)

    model.load_state_dict(torch.load('fcn_car_price_model_v2.pth', map_location='cpu'))
    model.eval()

    input_df = pd.DataFrame([feature_defaults])

    for key, value in car_features.items():
        if key in input_df.columns and key not in loaded_encoders:
            input_df.at[0, key] = value

    if 'description' in car_features and 'description_length' in input_df.columns:
        desc_len = len(str(car_features['description']))
        input_df.at[0, 'description_length'] = desc_len

    for col, le in loaded_encoders.items():
        if col in car_features:
            raw_val = str(car_features[col])
            known = set(le.classes_)
            if raw_val not in known:
                most_frequent = le.classes_[0]
                print(f"⚠️  Неизвестная категория '{raw_val}' в '{col}', заменяем на '{most_frequent}'")
                raw_val = most_frequent
            encoded = le.transform([raw_val])[0]
            input_df.at[0, col] = encoded
        else:
            pass

    input_df = input_df[feature_names]

    X_scaled = scaler.transform(input_df.astype(np.float32))
    X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

    with torch.no_grad():
        pred_log = model(X_tensor).numpy().squeeze()

    print("DEBUG pred_log (before clip):", pred_log)

    if not np.isfinite(pred_log):
        print("⚠️ pred_log не конечный (NaN/inf). Возвращаю NaN.")
        return float("nan")

    pred_log = np.clip(pred_log, log_min, log_max)

    pred_price = float(np.expm1(pred_log))
    return pred_price


In [ ]:
example_car = {
    'brand': 'AUDI',
    'bodyType': 'Седан',
    'color': '040001',
    'fuelType': 'бензин',
    'modelDate': 2015.0,
    'productionDate': 2016.0,
    'mileage': 50000,
    'numberOfDoors': 4.0,
    'engineDisplacement': 2.0,
    'enginePower': 180.0,
}

price = predict_car_price_fcn(example_car)
print(f"Предсказанная цена: {price:,.0f} руб.")


DEBUG pred_log (before clip): 14.056812
Предсказанная цена: 1,272,904 руб.


In [ ]:
test_examples_5 = [
    {
        "name": "1) Hyundai Solaris, популярный бюджетный седан",
        "car": {
            "brand": "HYUNDAI",
            "bodyType": "Седан",
            "color": "FFFFFF",
            "fuelType": "бензин",
            "modelDate": 2017.0,
            "productionDate": 2018.0,
            "mileage": 65_000,
            "numberOfDoors": 4.0,
            "engineDisplacement": 1.6,
            "enginePower": 123.0,
        },
    },
    {
        "name": "2) Toyota Camry, бизнес-седан среднего класса",
        "car": {
            "brand": "TOYOTA",
            "bodyType": "Седан",
            "color": "040001",
            "fuelType": "бензин",
            "modelDate": 2016.0,
            "productionDate": 2017.0,
            "mileage": 90_000,
            "numberOfDoors": 4.0,
            "engineDisplacement": 2.5,
            "enginePower": 181.0,
        },
    },
    {
        "name": "3) BMW 3 серии, премиум-седан, умеренный пробег",
        "car": {
            "brand": "BMW",
            "bodyType": "Седан",
            "color": "CACECB",
            "fuelType": "бензин",
            "modelDate": 2015.0,
            "productionDate": 2016.0,
            "mileage": 110_000,
            "numberOfDoors": 4.0,
            "engineDisplacement": 2.0,
            "enginePower": 184.0,
        },
    },
    {
        "name": "4) BMW X5, премиум-внедорожник",
        "car": {
            "brand": "BMW",
            "bodyType": "Внедорожник 5 дв.",
            "color": "CACECB",
            "fuelType": "дизель",
            "modelDate": 2015.0,
            "productionDate": 2016.0,
            "mileage": 130_000,
            "numberOfDoors": 5.0,
            "engineDisplacement": 3.0,
            "enginePower": 249.0,
        },
    },
    {
        "name": "5) KIA Sportage, популярный кроссовер",
        "car": {
            "brand": "KIA",
            "bodyType": "Внедорожник 5 дв.",
            "color": "0000CC",
            "fuelType": "бензин",
            "modelDate": 2018.0,
            "productionDate": 2019.0,
            "mileage": 55_000,
            "numberOfDoors": 5.0,
            "engineDisplacement": 2.0,
            "enginePower": 150.0,
        },
    },
]

for ex in test_examples_5:
    name = ex["name"]
    car = ex["car"]
    price_pred = predict_car_price_fcn(car)
    print(name)
    print(car)
    print(f"  → Предсказанная цена: {price_pred:,.0f} руб.\n")


⚠️  Неизвестная категория 'FFFFFF' в 'color', заменяем на '0000CC'
DEBUG pred_log (before clip): 13.701879
1) Hyundai Solaris, популярный бюджетный седан
{'brand': 'HYUNDAI', 'bodyType': 'Седан', 'color': 'FFFFFF', 'fuelType': 'бензин', 'modelDate': 2017.0, 'productionDate': 2018.0, 'mileage': 65000, 'numberOfDoors': 4.0, 'engineDisplacement': 1.6, 'enginePower': 123.0}
  → Предсказанная цена: 892,585 руб.

DEBUG pred_log (before clip): 14.024096
2) Toyota Camry, бизнес-седан среднего класса
{'brand': 'TOYOTA', 'bodyType': 'Седан', 'color': '040001', 'fuelType': 'бензин', 'modelDate': 2016.0, 'productionDate': 2017.0, 'mileage': 90000, 'numberOfDoors': 4.0, 'engineDisplacement': 2.5, 'enginePower': 181.0}
  → Предсказанная цена: 1,231,933 руб.

DEBUG pred_log (before clip): 14.049864
3) BMW 3 серии, премиум-седан, умеренный пробег
{'brand': 'BMW', 'bodyType': 'Седан', 'color': 'CACECB', 'fuelType': 'бензин', 'modelDate': 2015.0, 'productionDate': 2016.0, 'mileage': 110000, 'numberOfDoo